# Task 2

Требуется проверить гипотезу о равенстве средних на каждом уровне фактора с помощью модели однофакторного дисперсионного анализа.

## Calculate data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("iris.csv")
df["TotalArea"] = df["Sepal.Length"] * df["Sepal.Width"] + df["Petal.Length"] * df["Petal.Width"]
display(df)

splitted = np.array(list(map(lambda x: x[1], df.groupby("Species")["TotalArea"])))
splitted

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,TotalArea
0,5.1,3.5,1.4,0.2,setosa,18.13
1,4.9,3.0,1.4,0.2,setosa,14.98
2,4.7,3.2,1.3,0.2,setosa,15.30
3,4.6,3.1,1.5,0.2,setosa,14.56
4,5.0,3.6,1.4,0.2,setosa,18.28
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,32.06
146,6.3,2.5,5.0,1.9,virginica,25.25
147,6.5,3.0,5.2,2.0,virginica,29.90
148,6.2,3.4,5.4,2.3,virginica,33.50


array([[18.13, 14.98, 15.3 , 14.56, 18.28, 21.74, 16.06, 17.3 , 13.04,
        15.34, 20.28, 16.64, 14.54, 13.01, 23.44, 25.68, 21.58, 18.27,
        22.17, 19.83, 18.7 , 19.47, 16.76, 17.68, 16.7 , 15.32, 17.64,
        18.5 , 17.96, 15.36, 15.2 , 18.96, 21.47, 23.38, 15.49, 16.24,
        19.51, 17.78, 13.46, 17.64, 17.89, 10.74, 14.34, 18.46, 20.14,
        14.82, 19.7 , 15.  , 19.91, 16.78],
       [28.98, 27.23, 28.74, 17.85, 25.1 , 21.81, 28.31, 15.06, 25.12,
        19.5 , 13.5 , 24.  , 17.2 , 24.27, 20.92, 26.93, 23.55, 19.76,
        20.39, 18.29, 27.52, 22.28, 23.1 , 22.72, 24.15, 25.96, 25.76,
        28.6 , 24.15, 18.32, 17.38, 16.9 , 20.34, 24.36, 22.95, 27.6 ,
        27.82, 20.21, 22.13, 18.95, 19.58, 24.74, 19.88, 14.8 , 20.58,
        22.14, 21.99, 23.57, 16.05, 21.29],
       [35.79, 25.35, 33.69, 28.35, 32.26, 36.66, 19.9 , 32.51, 27.19,
        41.17, 31.  , 27.35, 31.95, 24.25, 28.48, 32.67, 29.4 , 44.  ,
        35.89, 20.7 , 35.19, 25.48, 34.96, 25.83, 34.08, 33.

## Формулировка гипотезы

$H_0$: Средняя площадь чашелистика + цветка равна у разных подвидов ириса

$H_1$: Средняя площадь чашелистика + цветка отличается у разных подвидов ириса

### Проверка библиотечным методом

In [2]:
from scipy.stats import f_oneway

f_oneway(*splitted)

F_onewayResult(statistic=133.29723648348485, pvalue=9.540732260908254e-34)

$\text{p-value} \ \ll 0.05 \Rightarrow H_0 \ \text{опровергнута в пользу} \ H_1$ 

### Реализация

In [3]:
from scipy.special import betainc

def anova(data: np.ndarray):
    I = data.size
    J = len(data)
    m = data.mean()
    s = data.var()
    
    def calc_SSBetweenGroups(data):
        l = 0
        for group in data:
            l += group.sum() ** 2 / len(group)  
        
        r = data.sum() ** 2 / I
        
        return l - r
        
    def calc_SSInGroups(data):
        l = (data ** 2).sum()
        
        r = 0
        for group in data:
            r += group.sum() ** 2 / len(group)  
        
        return l - r
    
    def f_cdf(x, a, b):
        return betainc(a / 2, b / 2, a * x / (a * x + b))
    
    SSBetweenGroups = calc_SSBetweenGroups(data)
    SSInGroups = calc_SSInGroups(data)
    
    DFBetweenGroups = J - 1
    DFInGroups = I - J

    MSBetweenGroups = SSBetweenGroups / DFBetweenGroups
    MSInGroups = SSInGroups / DFInGroups
    
    F = MSBetweenGroups / MSInGroups
    p_value = 1 - f_cdf(F, DFBetweenGroups, DFInGroups)
    return {
        "F-statistic": F,
        "p-value": p_value,
    }

anova(splitted)

{'F-statistic': 133.29723648348494, 'p-value': 1.1102230246251565e-16}

Мы видим, что значение p-value значительно меньше 0.05, таким образом, гипотеза $H_0$ опровергается, и мы говорим, что площадь не может не зависеть от сорта ириса.